In [1]:
import os

In [2]:
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction\\notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction'

In [4]:
from dataclasses import dataclass, field
from pathlib import Path

In [5]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    model_params: dict = field(default_factory=dict)

In [6]:
from src.constants import *
from src.utils.helper import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH, schema_filepath = SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            model_params = params,
            target_column = schema.name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from src import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        logger.info(f"Start prediction on {self.config.target_column}")
        rf = RandomForestClassifier(**self.config.model_params)
        logger.info(f"Model params:\n{rf.get_params()}")
        rf.fit(train_x, train_y)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-05-23 10:00:17,510: INFO: helper: yaml file: config\config.yaml loaded successfully]
[2025-05-23 10:00:17,517: INFO: helper: yaml file: params.yaml loaded successfully]
[2025-05-23 10:00:17,519: INFO: helper: yaml file: schema.yaml loaded successfully]
[2025-05-23 10:00:17,519: INFO: helper: created directory at: artifacts]
[2025-05-23 10:00:17,525: INFO: helper: created directory at: artifacts/model_trainer]
[2025-05-23 10:00:17,538: INFO: 3822362507: Start prediction on weather]
[2025-05-23 10:00:17,543: INFO: 3822362507: Model params:
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}]


d:\My\AboutData\Project\Weather-Prediction\weather-prediction-venv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
